## GuidedLDA что такое и зачем оно надо
*   Оригинал от автора библиотеки - https://medium.freecodecamp.org/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164
*   Репозиторий - https://github.com/vi3k6i5/GuidedLDA
*   Основная работа, которую использовали для создания GuidedLDA - http://www.umiacs.umd.edu/user.php?path=jags/pdfs/GuidedLDA.pdf


**Ключевое отличие от LDA**: можно усилить необходимые топики или разделить схожие, если корпус текстов не большой - при инициализации усиливаем какой-либо топик (см. параметр seed_confidence)

In [0]:
! pip install guidedlda
! pip install gensim
! pip install stop_words

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 2014-12-rbc.ru.csv to 2014-12-rbc.ru.csv
User uploaded file "2014-12-rbc.ru.csv" with length 22864554 bytes


In [3]:
import nltk
import re
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
import guidedlda
import warnings
import stop_words

from nltk.corpus import stopwords
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.utils import simple_preprocess
from nltk.tokenize import RegexpTokenizer
from gensim.models import CoherenceModel, LdaModel
from gensim.corpora import Dictionary
from stop_words import get_stop_words

nltk.download('stopwords') 
stemmer = SnowballStemmer('russian')
nltk.download('wordnet')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
data = pd.read_csv('2014-12-rbc.ru.csv', sep=',', encoding='utf-8')
data.columns = ['URL','date','topics','title','text']

In [5]:
data.head()

,URL,date,topics,title,text
0,http://top.rbc.ru/politics/01/12/2014/547bb1f8...,2014-12-01 00:11:32,NaN,ДНР и ЛНР просят заменить украинские банки абх...,До начала вооруженного конфликта в Донбассе ра...
1,http://www.rbc.ru/rbcfreenews/547bb630cbb20f09...,2014-12-01 00:28:34,NaN,Приморские города Крыма получат установки по о...,Приморские города Крыма получат установки по о...
2,http://www.rbc.ru/rbcfreenews/547bbd6dcbb20f53...,2014-12-01 00:59:25,NaN,Швейцарцы проголосовали против наращивания зол...,Граждане Швейцарии не поддержали на прошедшем ...
3,http://top.rbc.ru/economics/01/12/2014/547bc22...,2014-12-01 01:19:44,NaN,Стоимость барреля нефти опустилась ниже 69 дол...,Стоимость барреля нефти опустилась ниже $69\nА...
4,http://www.rbc.ru/rbcfreenews/547bc4bbcbb20fbc...,2014-12-01 01:30:36,NaN,На северо-востоке Подмосковья в ДТП погибли че...,На северо-востоке Подмосковья в дорожно-трансп...


In [0]:
# Фиксируем константы
np.random.seed(42)

#Stemmer
stemmer = SnowballStemmer('russian')

In [0]:
# Чистим текст
def clean_text(text):    
    text = re.sub("[^а-яА-Я]", ' ', str(text))
    text = re.sub("\'", "", text)
    text = re.sub('\s+', ' ', text)
    return text

In [0]:
data['cleaned_text'] = data['text'].apply(clean_text)

In [9]:
data.head()

,URL,date,topics,title,text,cleaned_text
0,http://top.rbc.ru/politics/01/12/2014/547bb1f8...,2014-12-01 00:11:32,NaN,ДНР и ЛНР просят заменить украинские банки абх...,До начала вооруженного конфликта в Донбассе ра...,До начала вооруженного конфликта в Донбассе ра...
1,http://www.rbc.ru/rbcfreenews/547bb630cbb20f09...,2014-12-01 00:28:34,NaN,Приморские города Крыма получат установки по о...,Приморские города Крыма получат установки по о...,Приморские города Крыма получат установки по о...
2,http://www.rbc.ru/rbcfreenews/547bbd6dcbb20f53...,2014-12-01 00:59:25,NaN,Швейцарцы проголосовали против наращивания зол...,Граждане Швейцарии не поддержали на прошедшем ...,Граждане Швейцарии не поддержали на прошедшем ...
3,http://top.rbc.ru/economics/01/12/2014/547bc22...,2014-12-01 01:19:44,NaN,Стоимость барреля нефти опустилась ниже 69 дол...,Стоимость барреля нефти опустилась ниже $69\nА...,Стоимость барреля нефти опустилась ниже Артем ...
4,http://www.rbc.ru/rbcfreenews/547bc4bbcbb20fbc...,2014-12-01 01:30:36,NaN,На северо-востоке Подмосковья в ДТП погибли че...,На северо-востоке Подмосковья в дорожно-трансп...,На северо востоке Подмосковья в дорожно трансп...


In [0]:
# Токенизируем 
tokenizer = RegexpTokenizer(r'\w+')
def tokenize(text):
    tokens = tokenizer.tokenize(text)
    return tokens

In [0]:
ru_stop = get_stop_words('ru')

# Удаялем стоп-слова
def stop_words(text):
    text = " ".join([i for i in text.lower().split() if i not in ru_stop])
    return text

In [0]:
# Лемматизируем
def lemmatize(text):    
    return stemmer.stem(WordNetLemmatizer().lemmatize(text))

In [0]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if len(token) >= 3:
            result.append(lemmatize_stemming(token))
    return result

In [14]:
%%time
data.loc[:,'stop_w_text'] = data['cleaned_text'].apply(stop_words)

CPU times: user 5.97 s, sys: 27 ms, total: 6 s
Wall time: 6 s


In [15]:
# Смотрим что получилось
print(data['stop_w_text'][0])

вооруженного конфликта донбассе работало общей сложности отделения банков сообщает газета известия прекращения деятельности возник определенный вакуум поскольку российские банки спешат прийти восток украины угрозы санкций сложившихся условиях власти днр лнр направили предложения абхазским банкам бояться санкций ес сша сухуме сомневаются сотрудничество целесообразным эмма тания национального банка абхазии сообщила неофициальные предложения донецка луганска приходили кажутся несерьезными поскольку копии письма стояли некоторые коммерческие банки словам реальной возможности работать рубежом абхазских финансовых организаций получении письма электронной почте сообщили сухум банке всерьез предложение рассматривали несмотря новороссию хочется поддержать чисто человечески непонятно насколько открытие филиалов экономически целесообразно председатель правления сухум банка даут агрба универсал банке письмо рассматривали посчитав речь шла спаме открыть филиал новороссии например днр подобран штат 

In [0]:
# Токенизируем
data.loc[:,'token_text'] = data['stop_w_text'].apply(tokenize)

In [18]:
data.head()

,URL,date,topics,title,text,cleaned_text,stop_w_text,token_text
0,http://top.rbc.ru/politics/01/12/2014/547bb1f8...,2014-12-01 00:11:32,NaN,ДНР и ЛНР просят заменить украинские банки абх...,До начала вооруженного конфликта в Донбассе ра...,До начала вооруженного конфликта в Донбассе ра...,вооруженного конфликта донбассе работало общей...,"[вооруженного, конфликта, донбассе, работало, ..."
1,http://www.rbc.ru/rbcfreenews/547bb630cbb20f09...,2014-12-01 00:28:34,NaN,Приморские города Крыма получат установки по о...,Приморские города Крыма получат установки по о...,Приморские города Крыма получат установки по о...,приморские города крыма получат установки опре...,"[приморские, города, крыма, получат, установки..."
2,http://www.rbc.ru/rbcfreenews/547bbd6dcbb20f53...,2014-12-01 00:59:25,NaN,Швейцарцы проголосовали против наращивания зол...,Граждане Швейцарии не поддержали на прошедшем ...,Граждане Швейцарии не поддержали на прошедшем ...,граждане швейцарии поддержали прошедшем рефере...,"[граждане, швейцарии, поддержали, прошедшем, р..."
3,http://top.rbc.ru/economics/01/12/2014/547bc22...,2014-12-01 01:19:44,NaN,Стоимость барреля нефти опустилась ниже 69 дол...,Стоимость барреля нефти опустилась ниже $69\nА...,Стоимость барреля нефти опустилась ниже Артем ...,стоимость барреля нефти опустилась артем филип...,"[стоимость, барреля, нефти, опустилась, артем,..."
4,http://www.rbc.ru/rbcfreenews/547bc4bbcbb20fbc...,2014-12-01 01:30:36,NaN,На северо-востоке Подмосковья в ДТП погибли че...,На северо-востоке Подмосковья в дорожно-трансп...,На северо востоке Подмосковья в дорожно трансп...,северо востоке подмосковья дорожно транспортно...,"[северо, востоке, подмосковья, дорожно, трансп..."


In [19]:
# Обрабатываем текст для LDA
processed_docs = data['stop_w_text'].map(tokenize)
processed_docs[0:10]

0    [вооруженного, конфликта, донбассе, работало, ...
1    [приморские, города, крыма, получат, установки...
2    [граждане, швейцарии, поддержали, прошедшем, р...
3    [стоимость, барреля, нефти, опустилась, артем,...
4    [северо, востоке, подмосковья, дорожно, трансп...
5    [скр, утверждают, причиной, преступления, стал...
6    [итоги, парламентских, выборов, молдавии, могл...
7    [социалистическая, партия, занимающая, первое,...
8    [российскую, спутниковую, группировку, пополни...
9    [член, комитета, транспорту, государственной, ...
Name: stop_w_text, dtype: object

In [0]:
# Для GuidedLDA нужна doc-term матрица в разреженном виде

from scipy.sparse import csr_matrix
docs = processed_docs

indptr = [0]
indices = []
data_csr = []
vocabulary = {}
for d in docs:
    for term in d:
        #print(f'term {term}')
        index = vocabulary.setdefault(term, len(vocabulary))
        #print(f'index {index} len_vocab {len(vocabulary)} ')
        indices.append(index)
        data_csr.append(1)
        #print(f'data {data}')
    indptr.append(len(indices))

csr_text_matrix = csr_matrix((data_csr, indices, indptr), dtype=int)

In [21]:
%%time
# Собираем словарь
dictionary = gensim.corpora.Dictionary(processed_docs)

CPU times: user 1.46 s, sys: 6.66 ms, total: 1.46 s
Wall time: 1.47 s


In [22]:
%%time
# а теперь Bag-of-words
BoW_corpus = [dictionary.doc2bow(text) for text in processed_docs]

CPU times: user 1.07 s, sys: 54.8 ms, total: 1.13 s
Wall time: 1.13 s


In [23]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 абхазии
1 абхазским
2 абхазских
3 агрба
4 адгур
5 аналитического
6 банка
7 банкам
8 банке
9 банки
10 банков


Надо попробовать стемминг (мб)

In [24]:
%%time
# пробуем LDA 
lda_model = gensim.models.LdaMulticore(BoW_corpus, num_topics=10, id2word=dictionary, passes=5, workers=2)

CPU times: user 34.6 s, sys: 3.43 s, total: 38 s
Wall time: 53.3 s


In [25]:
print("LDA Model:")
 
for idx in range(10):
    # Посмотрим первые 10 топиков
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))

LDA Model:
Topic #0: 0.006*"руб" + 0.005*"рбк" + 0.005*"россии" + 0.004*"млн" + 0.004*"тыс" + 0.004*"декабря" + 0.003*"компании" + 0.002*"заявил" + 0.002*"словам" + 0.002*"москвы"
Topic #1: 0.007*"чемпионат" + 0.005*"голы" + 0.005*"чемпионата" + 0.005*"матче" + 0.004*"цска" + 0.004*"й" + 0.004*"матч" + 0.004*"врезка" + 0.003*"тура" + 0.003*"спорт"
Topic #2: 0.003*"россии" + 0.003*"рфс" + 0.003*"команды" + 0.003*"нхл" + 0.003*"лиги" + 0.003*"место" + 0.003*"спорт" + 0.002*"сборной" + 0.002*"россия" + 0.002*"слова"
Topic #3: 0.006*"декабря" + 0.004*"россии" + 0.003*"украины" + 0.003*"рбк" + 0.003*"подборов" + 0.003*"заявил" + 0.003*"сша" + 0.003*"пресс" + 0.003*"млн" + 0.002*"президент"
Topic #4: 0.006*"россии" + 0.006*"украины" + 0.004*"компании" + 0.004*"рбк" + 0.004*"декабря" + 0.003*"заявил" + 0.003*"млрд" + 0.002*"ранее" + 0.002*"президент" + 0.002*"сша"
Topic #5: 0.015*"руб" + 0.008*"млрд" + 0.006*"россии" + 0.006*"декабря" + 0.005*"цб" + 0.004*"компании" + 0.004*"цены" + 0.004*"ба

In [26]:
for index, score in sorted(lda_model[BoW_corpus[0]], key=lambda tup: -1*tup[1]):

    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4999052584171295	 
Topic: 0.015*"руб" + 0.008*"млрд" + 0.006*"россии" + 0.006*"декабря" + 0.005*"цб" + 0.004*"компании" + 0.004*"цены" + 0.004*"банка" + 0.004*"евро" + 0.004*"рубля"

Score: 0.40573850274086	 
Topic: 0.006*"декабря" + 0.004*"россии" + 0.003*"украины" + 0.003*"рбк" + 0.003*"подборов" + 0.003*"заявил" + 0.003*"сша" + 0.003*"пресс" + 0.003*"млн" + 0.002*"президент"

Score: 0.09006073325872421	 
Topic: 0.004*"россии" + 0.004*"компании" + 0.003*"ес" + 0.003*"млрд" + 0.003*"рбк" + 0.003*"компания" + 0.002*"президент" + 0.002*"декабря" + 0.002*"путин" + 0.002*"заявил"


In [27]:
%%time
# Пробуем GuidedLDA
vocab = []
count = 0
for k, v in dictionary.iteritems():
    vocab.append(v)

word2id = dict((v, idx) for idx, v in enumerate(vocab))

# LDA без посева (выделения) топиков
model = guidedlda.GuidedLDA(n_topics=10, n_iter=100, random_state=7, refresh=20)
model.fit(csr_text_matrix)

INFO:guidedlda:n_documents: 7076
INFO:guidedlda:vocab_size: 89025
INFO:guidedlda:n_words: 1065122
INFO:guidedlda:n_topics: 10
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -14023440
INFO:guidedlda:<20> log likelihood: -10841296
INFO:guidedlda:<40> log likelihood: -10580925
INFO:guidedlda:<60> log likelihood: -10476055
INFO:guidedlda:<80> log likelihood: -10422982
INFO:guidedlda:<99> log likelihood: -10391885


CPU times: user 23.3 s, sys: 77.1 ms, total: 23.3 s
Wall time: 23.3 s


In [28]:
topic_word = model.topic_word_
n_top_words = 10
for i, topic_dist in enumerate(topic_word):
  topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
  print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: соответствующим водителя вырастить мордовией набирает влад заявить выиграет футбола объединилась
Topic 1: венгера финиш футболистам агуэро использовал индивидуальной анхеля группу находились коты
Topic 2: подобное республики собирается ближайшие роснефть увидели предприятия трое лидеров дать
Topic 3: республики внимании наращивания увидели частные роснефть металла взят организаций защиты
Topic 4: арена ближайшие банке канада витязь перепалки газом ослеживают ндс постановления
Topic 5: соответствующим некоторых евразийскую роснефть расизме рынке заместитель практическом автомобилей внешней
Topic 6: внимании массе ближайшие волна картеля возможного соответствующим сумму наращивания парламенте
Topic 7: соответствующим конфликта артем коммерческие коммуникациями долгов даут запретить водителя заместитель
Topic 8: выйти ближайшие соответствующим заместитель двое воды некоторых сообщение партия организаций
Topic 9: соответствующим роснефть возвращался служащие металла пролив государ

In [0]:
# GuidedLDA с выделенными топиками.
seed_topic_list = [['футбол', 'мяч', 'гол', 'стадион'],
                   ['банк', 'компания', 'цена', 'нефть', 'консорциум'],
                   ['путин', 'государство', 'страна', 'роснефть', 'лидер'],
                   ['ндс', 'рынок', 'дума', 'налог' ]]

In [30]:
%%time
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
  for word in st:
    seed_topics[word2id[word]] = t_id        

model = guidedlda.GuidedLDA(n_topics=10, n_iter=100, random_state=7, refresh=20)
        
model.fit(csr_text_matrix, seed_topics=seed_topics, seed_confidence=0.15)

INFO:guidedlda:n_documents: 7076
INFO:guidedlda:vocab_size: 89025
INFO:guidedlda:n_words: 1065122
INFO:guidedlda:n_topics: 10
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -14023068
INFO:guidedlda:<20> log likelihood: -10824291
INFO:guidedlda:<40> log likelihood: -10585316
INFO:guidedlda:<60> log likelihood: -10489457
INFO:guidedlda:<80> log likelihood: -10440603
INFO:guidedlda:<99> log likelihood: -10411071


CPU times: user 23.1 s, sys: 71.5 ms, total: 23.1 s
Wall time: 23.1 s


In [31]:
n_top_words = 10
topic_word = model.topic_word_

for i, topic_dist in enumerate(topic_word):
  topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
  print('Topic GuidedLDA {}: {}'.format(i, ' '.join(topic_words)))

Topic GuidedLDA 0: соответствующим водителя автогол коты влад финиш набирает выиграет вырастить агуэро
Topic GuidedLDA 1: венгера финиш футболистам агуэро анхеля использовал находились динамо группу индивидуальной
Topic GuidedLDA 2: республики увидели внимании роснефть частные наращивания металла подобное взят трое
Topic GuidedLDA 3: выйти соответствующим ближайшие конфликта заместитель сообщение коммерческие некоторых долгов запретить
Topic GuidedLDA 4: ближайшие банке канада воды перепалки недвижимости директор грузовика партия грузопассажирского
Topic GuidedLDA 5: соответствующим роснефть евразийскую годы рынке автомобилей внешней валентина отбиваться металла
Topic GuidedLDA 6: внимании массе наращивания например сумму возможного ближайшие парламенте консорциумы волна
Topic GuidedLDA 7: картеля далее декабря республики артем соответствующим металла китайских наращивания внимании
Topic GuidedLDA 8: соответствующим заместитель практическом некоторых евразийскую двое глава ближайшие ок